In [1]:
# Oanda API teszter

In [2]:
## Account

In [15]:
import pandas as pd

In [9]:
import cfg
import trader
import quant

In [10]:
print(f'{cfg.account.balance} nav:{cfg.account.NAV} margin:{cfg.account.marginAvailable}')

1512.1392 nav:1512.2025 margin:1457.3331


In [17]:
trades=[]
for t in cfg.account.trades:
    trade = dict(
    instrument=t.instrument,
    currentunits=t.currentUnits,
    unrealizedPL = t.unrealizedPL)
    trades.append(trade)
trades_df = pd.DataFrame(trades)
trades_df

,instrument,currentunits,unrealizedPL
0,EUR_USD,-373.0,0.3059
1,EUR_GBP,-371.0,0.0649
2,EUR_CHF,-367.0,0.2612
3,EUR_JPY,-368.0,-0.0161
4,EUR_CAD,366.0,-0.2377


In [4]:
print(f'{cfg.account.balance} nav:{cfg.account.NAV} margin:{cfg.account.marginAvailable}')

1512.1392 nav:1512.7256 margin:1466.2468


In [5]:
def check_breakeven_for_instrument(instrument: str, trades: list):
    all_breakeven = []
    for t in trades:
        if t.instrument == instrument:
            for o in cfg.account.orders:
                if o.id == t.stopLossOrderID:
                    all_breakeven.append(
                    (t.currentUnits > 0 ) and (o.price >=t.price)
                    or
                    (t.currentUnits <0 and o.price<=t.price))
    return all(all_breakeven)

In [6]:
check_breakeven_for_instrument('AUD_USD', cfg.account.trades)

True

In [7]:
for t in cfg.account.trades:
    print(t)
    break

id: '186741'
instrument: EUR_USD
price: 1.14454
openTime: '2022-02-07T15:18:02.366149067Z'
state: OPEN
initialUnits: -373.0
initialMarginRequired: 8.5388
currentUnits: -373.0
realizedPL: 0.0
unrealizedPL: 0.1417
marginUsed: 8.535
financing: 0.0
takeProfitOrderID: '186742'
stopLossOrderID: '186743'
trailingStopLossOrderID: '186744'


In [8]:
for o in cfg.account.orders:
    print(o)
    break

id: '183410'
createTime: '2022-01-26T09:38:07.001886322Z'
state: PENDING
type: TAKE_PROFIT
tradeID: '183409'
price: 0.71371
timeInForce: GTC
triggerCondition: DEFAULT


In [9]:
for t in cfg.account.trades:
    print(f'{t.instrument} {t.unrealizedPL}')

EUR_USD 0.5533
EUR_CHF -0.0121
USD_CHF 0.2413
AUD_USD 0.3439
AUD_JPY 0.4979
AUD_CAD -0.0643
EUR_GBP -0.0647


In [10]:
for p in cfg.account.positions:
    print(p)
    break

instrument: EUR_USD
pl: -388.9146
unrealizedPL: 0.6079
marginUsed: 8.2099
resettablePL: -388.9146
financing: -78.4801
commission: 0.0
guaranteedExecutionFees: 0.0
long:
  units: 0.0
  pl: -156.3461
  unrealizedPL: 0.0
  resettablePL: -156.3461
  financing: -79.6348
  guaranteedExecutionFees: 0.0
short:
  units: -364.0
  averagePrice: 1.12933
  tradeIDs:
  - '183576'
  pl: -232.5685
  unrealizedPL: 0.6079
  resettablePL: -232.5685
  financing: 1.1547
  guaranteedExecutionFees: 0.0


In [11]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.12766,
 'ask': 1.12779,
 'spread': 0.00013}